<a href="https://colab.research.google.com/github/jocelynbaduria/cmpe-297_SOTA/blob/main/Wav2Vec_Audio_Transcribing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ReadMe:

Reference: 

https://huggingface.co/models?search=wav2vec2

https://towardsdatascience.com/building-nlp-web-apps-with-gradio-and-hugging-face-transformers-59ce8ab4a319

https://github.com/chuachinhon/gradio_nlp/blob/main/notebooks/4.0_gradio_audio_text.ipynb

1. Import Libraries and install some module gradio, wandb and transformers.

2. Initialize  weights and biases (id = asr_5)

3. Load pre-trained Wav2Vec2 model and define the speech to text function. You can choose any model that will suits your needs from [here](https://huggingface.co/facebook/wav2vec2-base-960h)

4. Add Gradio app for testing.



### 1. Import Libraries and install some module gradio, wandb and transformers.


In [1]:
!pip install gradio -q
# Install wandb for experiment tracking
!pip install wandb --upgrade -q
!pip install -q git+https://github.com/huggingface/transformers.git

     |████████████████████████████████| 2.0 MB 5.0 MB/s 
     |████████████████████████████████| 206 kB 45.7 MB/s 
     |████████████████████████████████| 1.9 MB 50.9 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 961 kB 72.2 MB/s 
     |████████████████████████████████| 3.5 MB 50.9 MB/s 
     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 139 kB 60.3 MB/s 
     |████████████████████████████████| 180 kB 63.7 MB/s 
     |████████████████████████████████| 97 kB 5.4 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3 MB 5.3 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 895 kB 71.6 MB/s 
     |████████████████████████████████| 596 kB 73.1 MB/s 


In [3]:
import gradio as gr
import librosa
import soundfile as sf
import torch
# import warnings

# from transformers import AutoTokenizer, Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Tokenizer
from transformers import Wav2Vec2ForMaskedLM, Wav2Vec2Tokenizer

# warnings.filterwarnings("ignore")

### 2. Initialize  weights and biases (id = asr_5)

In [4]:
import torch
# Other imports 
import wandb
wandb.login()
# from wandb.keras import WandbCallback
from tqdm import tqdm
wandb.init(project="Speech2Text", entity="jocelynbaduria", id="asr_5")
# 2. Save model inputs and hyperparameters
config = wandb.config
config.learning_rate = 0.01

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jocelynbaduria (use `wandb login --relogin` to force relogin)


### 3. Load pre-trained Wav2Vec2 model and define the speech to text function. You can choose any model that will suits your needs from [here](https://huggingface.co/facebook/wav2vec2-base-960h)

In [6]:
# load wav2vec2 tokenizer and model
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2ForMaskedLM.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")

#define speech to text function
def asr_transcribe(audio_file):
  transcript = ""

  # Stream over 20seconds chunks
  stream = librosa.stream(
      audio_file.name, block_length=20, frame_length=16000, hop_length=16000
  )

  for speech in stream:
    if len(speech.shape) > 1:
      speech = speech[:, 0] + speech[:, 1]

    input_values = tokenizer(speech, return_tensors="pt").input_values
    logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    transcript += transcription.lower() + " "

  return transcript

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:423: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,


Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1396: FutureWarning: The class `Wav2Vec2ForMaskedLM` is deprecated. Please use `Wav2Vec2ForCTC` instead.
  "The class `Wav2Vec2ForMaskedLM` is deprecated. Please use `Wav2Vec2ForCTC` instead.", FutureWarning
Some weights of Wav2Vec2ForMaskedLM were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 4. Add Gradio app for testing.

In [7]:
gradio_asr = gr.Interface(
    fn=asr_transcribe,
    title="Speech to Text with Wav2Vec Hugging Face",
    description="Upload an audio clip, and let Transcribe the word",
    inputs=gr.inputs.Audio(label="Upload Audio File", type="file"),
    outputs=gr.outputs.Textbox(label="Auto-Transcription"),
)

In [8]:
gradio_asr.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://32200.gradio.app


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://32200.gradio.app')